In [1]:
import pandas as pd
from dateutil import parser
import requests
import sqlite3
import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [3]:
c.execute("""SELECT * FROM Matches;""")
matches = pd.DataFrame(c.fetchall())
matches.columns = [x[0] for x in c.description]

In [4]:
berlin_lat = '52.520008'
berlin_long = '13.404954'

In [5]:
api_key = '7101c14ab733bd8ed9e205263525841b'

In [6]:
class Weather():

    
    def __init__(self, dataframe, year, lat, long, DarkSky_API):
        self.dataframe = dataframe
        self.year = year
        self.lat = lat
        self.long = long
        self.DarkSky_API = DarkSky_API

    
    def get_rain(self):
        """Time is timestamp, everything must be a string. Returns a list of DarkSky
        information. If you've already run this before, you can comment out the API
        call and reference a local file instead to save on API calls. Otherwise comment
        out the read_csv line. Executed within the rainy_days function."""
    #     dark_sky = []
    #     for days in time:
    #         resp = requests.get('https://api.darksky.net/forecast/' + self.DarkSky_API + '/' + self.lat + ',' + self.long + ',' + days + '?exclude=flags,hourly,daily,alerts')
    #         dark_sky.append(resp.json()['currently'])
        dark_sky = pd.read_csv('weather.csv') # Comment out this line if you don't have the data already
        return dark_sky

    
    def rainy_days(self):
        gameDate = []

        for i in range(len(self.dataframe['Date'])):
            dt = parser.parse(self.dataframe['Date'][i])
            gameDate.append(dt)

        self.dataframe['gamedate'] = gameDate
        matches_year = matches[matches["Season"] == self.year].reset_index()
        dates = list(set(matches_year['gamedate']))

        timestamp = []
        for date in range(len(dates)):
            stamp = str(matches_year['gamedate'][date].timestamp())
            timestamp.append(stamp[:-2])

        the_rain = Weather(self.lat, self.long, 2011, timestamp, self.DarkSky_API)
            
        ds_df = pd.DataFrame(the_rain.get_rain())

        df_rain = pd.DataFrame(
            {'rainy': ds_df['icon'],
             'date': ds_df['time'],
            })

        for day in range(len(ds_df['time'])):
            ds_df['time'][day] = datetime.datetime.utcfromtimestamp(ds_df['time'][day]).strftime('%Y-%m-%d')

        rain = pd.DataFrame(columns=['date', 'raining'])

        rain['date'] = ds_df['time']

        for i in range(len(ds_df['precipType'])):
            if ds_df['precipType'][i] == 'rain':
                rain.raining[i] = True
            else:
                rain.raining[i] = False

        rain_year = []
        for i in range(len(matches_year['Date'])):
            if matches_year['Date'][i] in list(rain.date[rain['raining'] == True]):
                rain_year.append(True)
            else:
                rain_year.append(False)

        matches_year['rain'] = rain_year

        result = matches_year[['Match_ID','Date','AwayTeam', 'HomeTeam', 'FTR', 'rain']]

        return result

In [7]:
season = Weather(matches, 2011, berlin_lat, berlin_long, api_key)

In [8]:
rainy_wins = season.rainy_days()[season.rainy_days()['rain'] == True].reset_index()

In [9]:
rainy_wins

,index,Match_ID,Date,AwayTeam,HomeTeam,FTR,rain
0,8,1100,2011-09-18,Bayern Munich,Schalke 04,A,True
1,18,1110,2011-12-17,Dortmund,Freiburg,A,True
2,26,1118,2011-09-18,Dortmund,Hannover,H,True
3,43,1135,2012-05-06,Dresden,Frankfurt FSV,D,True
4,66,1158,2012-05-06,Duisburg,Fortuna Dusseldorf,D,True
5,76,1168,2012-05-06,Ein Frankfurt,Karlsruhe,H,True
6,127,1219,2011-08-05,Hamburg,Dortmund,H,True
7,157,1249,2011-12-09,Karlsruhe,Duisburg,H,True
8,192,1284,2011-12-17,Nurnberg,Leverkusen,A,True
9,211,1303,2011-12-09,Schalke 04,Hertha,A,True


# Wins in the Rain

In [10]:
class Winning_in_the_Rain():

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def get_rain_wins(self):
        raining_wins = pd.concat([self.dataframe, pd.get_dummies(self.dataframe['FTR'])], axis = 1).reset_index()

        away_games = pd.DataFrame(
            {'away rain games': raining_wins['AwayTeam'].value_counts()    })
        home_games = pd.DataFrame(
            {'home rain games': raining_wins['HomeTeam'].value_counts()    })

        raining_wins['winner'] = 'tie game'

        for i in range(len(raining_wins['A'])):
            if raining_wins['A'][i] == 1:
                raining_wins['winner'][i] = raining_wins['AwayTeam'][i]
            if raining_wins['H'][i] == 1:
                raining_wins['winner'][i] = raining_wins['HomeTeam'][i]        

        total_wins = pd.DataFrame(
            {'wins': raining_wins['winner'].value_counts()
            })

        appearances = pd.concat([away_games, home_games, total_wins], axis = 1)
        appearances['appearances'] = 0
        appearances = appearances.fillna(0)

        for i in range(len(appearances['home rain games'])):
            appearances['appearances'][i] = appearances['home rain games'][i] + appearances['away rain games'][i]


        winning_rain_pct = []
        for i in range(len(appearances['home rain games'])):
            if appearances['appearances'][i] == 0:
                winning_rain_pct.append(0)
            else:
                winning_rain_pct.append(round(float(appearances['wins'][i]) / float(appearances['appearances'][i]), 2))

        appearances['winning_rain_pct'] = winning_rain_pct

        appearances = appearances.fillna('0').drop(['home rain games', 'away rain games'], axis=1)
        return appearances

season2011 = Winning_in_the_Rain(rainy_wins)
season2011.get_rain_wins()

,wins,appearances,winning_rain_pct
Aachen,1.0,3,0.33
Aston Villa,0.0,1,0.00
Augsburg,0.0,1,0.00
Bayern Munich,1.0,1,1.00
Blackburn,0.0,1,0.00
Bochum,1.0,3,0.33
Bolton,0.0,2,0.00
Braunschweig,1.0,2,0.50
Chelsea,0.0,2,0.00
Cottbus,1.0,1,1.00
